In [797]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [798]:
path="D:/Python/JantaHack"

In [799]:
train=pd.read_csv(path+"/"+"train_8wry4cB.csv")
test=pd.read_csv(path+"/"+"test_Yix80N0.csv")
sample_submission=pd.read_csv(path+"/"+"sample_submission_opxHi4g.csv")

In [800]:
train["Gender_Num"]=np.where(train["gender"]=="female",1,0)

In [801]:
train.drop(["gender"],axis=1,inplace=True)

In [802]:
train["Temp"]="train"
test["Temp"]="test"
test["Gender_Num"]=0

In [803]:
combi=pd.concat([train,test],axis=0, ignore_index=True)

In [804]:
print(combi.dtypes)
combi.head(6)

Gender_Num      int64
ProductList    object
Temp           object
endTime        object
session_id     object
startTime      object
dtype: object


,Gender_Num,ProductList,Temp,endTime,session_id,startTime
0,1,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,train,15/12/14 18:12,u16159,15/12/14 18:11
1,0,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,train,16/12/14 14:41,u10253,16/12/14 14:35
2,1,A00002/B00001/C00020/D16944/,train,01/12/14 15:58,u19037,01/12/14 15:58
3,1,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,train,23/11/14 3:00,u14556,23/11/14 2:57
4,0,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,train,17/12/14 16:46,u24295,17/12/14 16:44
5,1,A00002/B00002/C00081/D25039/,train,12/12/14 15:31,u20964,12/12/14 15:31


In [805]:
spliting=combi["ProductList"].str.split(";|/",expand=True)

In [806]:
modify=spliting.stack().groupby(level=0).apply(lambda x: x.unique().tolist())

In [807]:
Updated=list(modify)

In [808]:
iter1=pd.DataFrame(Updated)

In [809]:
### Taking the sum of unique Category in each row ##
result=[]
for i in iter1.index:
    result.append(iter1.iloc[i].str.contains("A").sum())

iter1["Num_Of_Cat"]=result

In [810]:
### Taking the sum of unique Sub Category in each row ##
result=[]
for i in iter1.index:
    result.append(iter1.iloc[i].str.contains("B").sum())

iter1["Num_Of_Sub_Cat"]=result

In [811]:
### Taking the sum of unique Sub Sub Category in each row ##
result=[]
for i in iter1.index:
    result.append(iter1.iloc[i].str.contains("C").sum())

iter1["Num_Of_Sub_Sub_cat"]=result

In [812]:
### Taking the sum of unique Product in each row ##
result=[]
for i in iter1.index:
    result.append(iter1.iloc[i].str.contains("D").sum())

iter1["Num_Of_Products"]=result

In [813]:
#### Naming the columns
iter1["Category"]=iter1[0]
iter1["Sub_Cat"]=iter1[1]
iter1["Sub_Sub_Cat"]=iter1[2]
iter1["Product"]=iter1[3]

In [814]:
#### Keeping only those which are required #####
iter2=iter1[["Category","Sub_Cat","Sub_Sub_Cat","Product","Num_Of_Cat","Num_Of_Sub_Cat","Num_Of_Sub_Sub_cat","Num_Of_Products"]]

In [815]:
del iter1

In [816]:
print(iter2.shape)
combi.shape

(15000, 8)


(15000, 6)

In [817]:
combi.drop(["ProductList"],axis=1, inplace=True)

In [818]:
combine=pd.concat([combi,iter2],axis=1)

In [819]:
import datetime
from datetime import timedelta, time, datetime
import time

In [820]:
def try_parsing_date(text):
    for fmt in ("%d/%m/%y %H:%M", "%d/%m/%Y %H:%M"):
            return datetime.strptime(text, fmt)

In [821]:
combine["startTime"]=combine["startTime"].apply(try_parsing_date)
combine["endTime"]=combine["endTime"].apply(try_parsing_date)

In [822]:
combine["start_seconds"] = [time.mktime(t.timetuple()) for t in combine["startTime"]]

In [823]:
combine["end_seconds"]= [time.mktime(t.timetuple()) for t in combine["endTime"]]

In [824]:
combine["Session_Seconds"]= round(combine["end_seconds"]-combine["start_seconds"])

In [825]:
combine["Start_Hour"]=combine["startTime"].dt.hour

In [826]:
combine.to_csv("check.csv",index=False)

In [827]:
combine.describe(percentiles=[0.01,0.05,0.95,0.99]).T

,count,mean,std,min,1%,5%,50%,95%,99%,max
Gender_Num,15000.0,5.461333e-01,4.978838e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Num_Of_Cat,15000.0,1.039800e+00,2.022013e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00
Num_Of_Sub_Cat,15000.0,1.153667e+00,4.493699e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,7.000000e+00
Num_Of_Sub_Sub_cat,15000.0,1.354867e+00,7.982766e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,5.000000e+00,1.300000e+01
Num_Of_Products,15000.0,2.230333e+00,2.340921e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00,1.200000e+01,4.300000e+01
start_seconds,15000.0,1.417720e+09,1.028639e+06,1.415904e+09,1.415941e+09,1.416034e+09,1.417873e+09,1.419158e+09,1.419257e+09,1.419273e+09
end_seconds,15000.0,1.417727e+09,1.027842e+06,1.415904e+09,1.415942e+09,1.416036e+09,1.417879e+09,1.419160e+09,1.419257e+09,1.419273e+09
Session_Seconds,15000.0,7.821396e+03,1.128470e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.800000e+02,7.321200e+03,3.045120e+06
Start_Hour,15000.0,1.419253e+01,5.106480e+00,0.000000e+00,0.000000e+00,7.000000e+00,1.400000e+01,2.200000e+01,2.300000e+01,2.300000e+01


In [828]:
numeric_col= combine[["Num_Of_Cat","Num_Of_Sub_Cat","Num_Of_Sub_Sub_cat","Num_Of_Products","Session_Seconds","Start_Hour"]]
for i in numeric_col:
    combine.loc[combine[i]>combine[i].quantile(0.99),i]=combine[i].quantile(0.99)
    combine.loc[combine[i]<combine[i].quantile(0.01),i]=combine[i].quantile(0.01)
combine.describe(percentiles=[0.01,0.05,0.95,0.99]).T

,count,mean,std,min,1%,5%,50%,95%,99%,max
Gender_Num,15000.0,5.461333e-01,4.978838e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Num_Of_Cat,15000.0,1.038467e+00,1.923264e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00
Num_Of_Sub_Cat,15000.0,1.147267e+00,4.101915e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00
Num_Of_Sub_Sub_cat,15000.0,1.345733e+00,7.363681e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,5.000000e+00,5.000000e+00
Num_Of_Products,15000.0,2.181467e+00,1.983095e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00,1.200000e+01,1.200000e+01
start_seconds,15000.0,1.417720e+09,1.028639e+06,1.415904e+09,1.415941e+09,1.416034e+09,1.417873e+09,1.419158e+09,1.419257e+09,1.419273e+09
end_seconds,15000.0,1.417727e+09,1.027842e+06,1.415904e+09,1.415942e+09,1.416036e+09,1.417879e+09,1.419160e+09,1.419257e+09,1.419273e+09
Session_Seconds,15000.0,1.843760e+02,7.862300e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.800000e+02,7.320012e+03,7.321200e+03
Start_Hour,15000.0,1.419253e+01,5.106480e+00,0.000000e+00,0.000000e+00,7.000000e+00,1.400000e+01,2.200000e+01,2.300000e+01,2.300000e+01


In [829]:
combine["Part_Of_Day"] = 0

In [830]:
combine.loc[(combine["Start_Hour"] >= 0 )& (combine["Start_Hour"]< 12),"Part_Of_Day"]="Morning"
combine.loc[(combine["Start_Hour"]>= 12) & (combine["Start_Hour"]< 18),"Part_Of_Day"]="Noon"
combine.loc[combine["Start_Hour"]>= 18 ,"Part_Of_Day"]="Evening"

In [831]:
combine.dtypes

Gender_Num                     int64
Temp                          object
endTime               datetime64[ns]
session_id                    object
startTime             datetime64[ns]
Category                      object
Sub_Cat                       object
Sub_Sub_Cat                   object
Product                       object
Num_Of_Cat                   float64
Num_Of_Sub_Cat               float64
Num_Of_Sub_Sub_cat           float64
Num_Of_Products              float64
start_seconds                float64
end_seconds                  float64
Session_Seconds              float64
Start_Hour                   float64
Part_Of_Day                   object
dtype: object

In [832]:
k=combine["Sub_Cat"].value_counts()
for i in k.axes[0][0:5]:
    varname='Sub_Cat_'+i.replace(',','_').replace(' ','_')
    combine[varname]=np.where(combine['Sub_Cat']==i,1,0)
del combine["Sub_Cat"]

In [833]:
combine["Sub_Sub_Cat"].value_counts()

C00007    1373
C00002     673
C00003     652
C00006     521
C00051     501
C00048     480
C00005     466
C00014     434
C00028     365
C00010     363
C00059     317
C00016     310
C00018     304
C00004     274
C00066     257
C00009     254
C00044     250
C00021     248
C00019     231
C00046     217
C00036     212
C00012     186
C00074     174
C00053     170
C00079     156
C00032     144
C00075     137
C00020     135
C00123     128
C00043     126
          ... 
C00368       1
C00316       1
C00384       1
C00378       1
C00407       1
C00357       1
C00279       1
C00381       1
C00339       1
C00390       1
C00204       1
C00191       1
C00262       1
C00372       1
C00399       1
C00188       1
C00366       1
C00313       1
C00295       1
C00439       1
C00413       1
C00395       1
C00199       1
C00369       1
C00141       1
C00235       1
C00124       1
C00382       1
C00240       1
C00408       1
Name: Sub_Sub_Cat, Length: 356, dtype: int64

In [834]:
k=combine["Sub_Sub_Cat"].value_counts()
for i in k.axes[0][0:5]:
    varname='Sub_Sub_Cat_'+i.replace(',','_').replace(' ','_')
    combine[varname]=np.where(combine['Sub_Sub_Cat']==i,1,0)
del combine["Sub_Sub_Cat"]

In [835]:
k=combine["Product"].value_counts()
for i in k.axes[0][0:5]:
    varname='Product_'+i.replace(',','_').replace(' ','_')
    combine[varname]=np.where(combine['Product']==i,1,0)
del combine["Product"]

In [836]:
dummy=pd.get_dummies(combine[["Category","Part_Of_Day"]],drop_first=False)

In [837]:
combine.columns

Index(['Gender_Num', 'Temp', 'endTime', 'session_id', 'startTime', 'Category',
       'Num_Of_Cat', 'Num_Of_Sub_Cat', 'Num_Of_Sub_Sub_cat', 'Num_Of_Products',
       'start_seconds', 'end_seconds', 'Session_Seconds', 'Start_Hour',
       'Part_Of_Day', 'Sub_Cat_B00002', 'Sub_Cat_B00001', 'Sub_Cat_B00003',
       'Sub_Cat_B00012', 'Sub_Cat_B00004', 'Sub_Sub_Cat_C00007',
       'Sub_Sub_Cat_C00002', 'Sub_Sub_Cat_C00003', 'Sub_Sub_Cat_C00006',
       'Sub_Sub_Cat_C00051', 'Product_D00266', 'Product_D01034',
       'Product_D20430', 'Product_D00232', 'Product_D19326'],
      dtype='object')

In [838]:
mydata=pd.concat([combine,dummy],axis=1)

In [839]:
mydata_2=mydata.drop(["endTime","session_id", "startTime", "Category", "Part_Of_Day","start_seconds", "end_seconds","Start_Hour"],axis=1)

In [840]:
mydata_2.columns

Index(['Gender_Num', 'Temp', 'Num_Of_Cat', 'Num_Of_Sub_Cat',
       'Num_Of_Sub_Sub_cat', 'Num_Of_Products', 'Session_Seconds',
       'Sub_Cat_B00002', 'Sub_Cat_B00001', 'Sub_Cat_B00003', 'Sub_Cat_B00012',
       'Sub_Cat_B00004', 'Sub_Sub_Cat_C00007', 'Sub_Sub_Cat_C00002',
       'Sub_Sub_Cat_C00003', 'Sub_Sub_Cat_C00006', 'Sub_Sub_Cat_C00051',
       'Product_D00266', 'Product_D01034', 'Product_D20430', 'Product_D00232',
       'Product_D19326', 'Category_A00001', 'Category_A00002',
       'Category_A00003', 'Category_A00004', 'Category_A00005',
       'Category_A00006', 'Category_A00007', 'Category_A00008',
       'Category_A00009', 'Category_A00010', 'Category_A00011',
       'Part_Of_Day_Evening', 'Part_Of_Day_Morning', 'Part_Of_Day_Noon'],
      dtype='object')

In [841]:
training=mydata_2.loc[mydata_2["Temp"]=="train"]
testing=mydata_2.loc[mydata_2["Temp"]=="test"]

In [842]:
training.drop(["Temp"],axis=1,inplace=True)
testing.drop(["Temp","Gender_Num"],axis=1,inplace=True)

In [843]:
from sklearn.model_selection import train_test_split

In [844]:
x_train=training.drop(["Gender_Num"],axis=1)
y_train=training["Gender_Num"]

In [845]:
np.random.seed(324)

In [846]:
X_train,X_test,Y_train,Y_test=train_test_split(x_train,y_train,test_size=0.30,stratify=y_train)

In [847]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [848]:
from xgboost.sklearn import XGBClassifier

In [849]:
from sklearn.model_selection import RandomizedSearchCV,train_test_split,GridSearchCV

In [850]:
param_dist = {
              "max_depth": [2,3,4,5,6,8],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000],
    'scale_pos_weight':[2,3,4,5,6,7,8,9]
    
              }

In [851]:
clf=XGBClassifier(objective='binary:logistic')

In [852]:
n_iter=10

random_search=RandomizedSearchCV(clf,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist)

In [853]:
random_search.fit(X_train,Y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.2min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'max_depth': [2, 3, 4, 5, 6, 8], 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5], 'min_child_weight': [4, 5, 6], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'reg_alpha': [1e-05, 0.01, 0.1, 1, 100], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'n_estimators': [100, 500, 700, 1000], 'scale_pos_weight': [2, 3, 4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=No

In [854]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.820 (std: 0.01759)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 2, 'reg_alpha': 1e-05, 'n_estimators': 100, 'min_child_weight': 6, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.4, 'colsample_bytree': 0.7}

Model with rank: 2
Mean validation score: 0.818 (std: 0.01700)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 3, 'reg_alpha': 0.1, 'n_estimators': 1000, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.7}

Model with rank: 3
Mean validation score: 0.817 (std: 0.01834)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 6, 'reg_alpha': 1e-05, 'n_estimators': 700, 'min_child_weight': 4, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.8}

Model with rank: 4
Mean validation score: 0.813 (std: 0.01757)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 5, 'reg_alpha': 0.01, 'n_estimators': 100, 'min_child_weight': 6, 'max_depth': 2, 'learning_rate': 0.05, 'g

In [858]:
xgb_best=XGBClassifier(subsample=0.6,scale_pos_weight=2,reg_alpha=1e-05,n_estimators=100,min_child_weight=6,
                       max_depth=5,learning_rate=0.05,gamma=0.4,colsample_bytree=0.7)

In [860]:
xgb_best.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=6, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=2, seed=None,
       silent=None, subsample=0.6, verbosity=1)

In [861]:
p=xgb_best.predict_proba(X_test)[:,1]

In [862]:
from sklearn.metrics import roc_auc_score

In [863]:
roc_auc_score(Y_test,p)

0.831654747738892

In [877]:
sample_submission["gender"]= xgb_best.predict_proba(testing)[:,1]

In [878]:
sample_submission["gender"]=np.where(sample_submission["gender"]>0.50,"female","male")

In [879]:
sample_submission.to_csv("D:/Python/JantaHack/sample_submission.csv",header =True, index=False)

In [ ]:
### XGB is the final model for this Hackathon, model stacking woudl imporve the accuracy of the model ###

In [ ]:
### Though catboost i giving good accuracy in training but not on the testing data ######

In [901]:
param_dist = {
              "num_leaves": [27,31,61,81,127,197],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "bagging_freq":[3,9,15,17,23,31],
    "max_depth": [2,3,4,5,6,8],
              "boosting_type":['gbdt','dart'],
 "min_data":[300,400,500,550,650],
               "max_bin":[5,10,12,18,20],
              "is_unbalance":[True , False],
    "lambda_l1":[0,10,20,30,40],
    'bagging_fraction':[0.5,0.7,0.8,0.9]
    
              }

In [902]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='Binary',random_state=5)

In [903]:
n_iter=10

random_search_2=RandomizedSearchCV(lgbm,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist)

In [904]:
random_search_2.fit(X_train,Y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.1s finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective='Binary',
        random_state=5, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'num_leaves': [27, 31, 61, 81, 127, 197], 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5], 'bagging_freq': [3, 9, 15, 17, 23, 31], 'max_depth': [2, 3, 4, 5, 6, 8], 'boosting_type': ['gbdt', 'dart'], 'min_data': [300, 400, 500, 550, 650], 'max_bin': [5, 10, 12, 18, 20], 'is_unbalance': [True, False], 'lambda_l1': [0, 10, 20, 30, 40], 'bagging_fraction': [0.5, 0.7, 0.8, 0.9]},
          pre_dispatch='2*n_

In [905]:
report(random_search_2.cv_results_,5)

Model with rank: 1
Mean validation score: 0.812 (std: 0.01539)
Parameters: {'num_leaves': 61, 'min_data': 300, 'max_depth': 3, 'max_bin': 5, 'learning_rate': 0.1, 'lambda_l1': 30, 'is_unbalance': True, 'boosting_type': 'gbdt', 'bagging_freq': 15, 'bagging_fraction': 0.9}

Model with rank: 2
Mean validation score: 0.811 (std: 0.01657)
Parameters: {'num_leaves': 61, 'min_data': 300, 'max_depth': 5, 'max_bin': 5, 'learning_rate': 0.01, 'lambda_l1': 0, 'is_unbalance': False, 'boosting_type': 'dart', 'bagging_freq': 31, 'bagging_fraction': 0.9}

Model with rank: 3
Mean validation score: 0.811 (std: 0.01858)
Parameters: {'num_leaves': 197, 'min_data': 300, 'max_depth': 3, 'max_bin': 12, 'learning_rate': 0.5, 'lambda_l1': 10, 'is_unbalance': False, 'boosting_type': 'gbdt', 'bagging_freq': 15, 'bagging_fraction': 0.5}

Model with rank: 4
Mean validation score: 0.810 (std: 0.01748)
Parameters: {'num_leaves': 81, 'min_data': 300, 'max_depth': 5, 'max_bin': 20, 'learning_rate': 0.5, 'lambda_l1': 

In [911]:
lgbm_best=LGBMClassifier(num_leaves=61,min_data=300,max_depth=3,max_bin=5,learning_rate=0.1,lambda_l1=30,is_unbalance=True,bossting_type='gbdt',bagging_freq=15,bagging_fraction=0.9)

In [914]:
lgbm_best.fit(X_train,Y_train)

LGBMClassifier(bagging_fraction=0.9, bagging_freq=15, boosting_type='gbdt',
        bossting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', is_unbalance=True, lambda_l1=30,
        learning_rate=0.1, max_bin=5, max_depth=3, min_child_samples=20,
        min_child_weight=0.001, min_data=300, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=61, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [915]:
p2=lgbm_best.predict_proba(X_test)[:,1]

In [916]:
roc_auc_score(Y_test,p2)

0.8216964071546489

In [917]:
sample_submission["gender"]= lgbm_best.predict_proba(testing)[:,1]

In [918]:
sample_submission["gender"]=np.where(sample_submission["gender"]>0.50,"female","male")

In [919]:
sample_submission.to_csv("D:/Python/JantaHack/sample_submission.csv",header =True, index=False)

In [ ]:
from catboost import CatBoostClassifier

In [950]:
model = CatBoostClassifier(class_weights=[1,2],eval_metric='AUC',loss_function="Logloss",random_seed = 23)
parameters = {'depth'         : [4,6,8,10,12],
                  'learning_rate' : [0.01,0.02,0.05,0.1],
                  'iterations'    : [500,1000,1500,2000,2500,3000]
                 }

In [951]:
random_search_3= RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 2, n_iter = 10, n_jobs=-1)

In [952]:
random_search_3.fit(X_train,Y_train)

0:	total: 3.35ms	remaining: 3.35s
1:	total: 7.6ms	remaining: 3.79s
2:	total: 11.9ms	remaining: 3.97s
3:	total: 14.9ms	remaining: 3.72s
4:	total: 18.1ms	remaining: 3.6s
5:	total: 22.1ms	remaining: 3.66s
6:	total: 25.7ms	remaining: 3.65s
7:	total: 28.6ms	remaining: 3.55s
8:	total: 32.1ms	remaining: 3.53s
9:	total: 34.9ms	remaining: 3.45s
10:	total: 37.6ms	remaining: 3.38s
11:	total: 40.4ms	remaining: 3.33s
12:	total: 43ms	remaining: 3.26s
13:	total: 45.9ms	remaining: 3.23s
14:	total: 48.7ms	remaining: 3.2s
15:	total: 51.2ms	remaining: 3.15s
16:	total: 54.1ms	remaining: 3.13s
17:	total: 57ms	remaining: 3.11s
18:	total: 59.9ms	remaining: 3.09s
19:	total: 63ms	remaining: 3.09s
20:	total: 65.8ms	remaining: 3.07s
21:	total: 68.7ms	remaining: 3.05s
22:	total: 71.6ms	remaining: 3.04s
23:	total: 74.1ms	remaining: 3.01s
24:	total: 76.6ms	remaining: 2.99s
25:	total: 79.3ms	remaining: 2.97s
26:	total: 82ms	remaining: 2.96s
27:	total: 84.7ms	remaining: 2.94s
28:	total: 87.3ms	remaining: 2.92s
29:	to

276:	total: 887ms	remaining: 2.31s
277:	total: 890ms	remaining: 2.31s
278:	total: 893ms	remaining: 2.31s
279:	total: 896ms	remaining: 2.3s
280:	total: 898ms	remaining: 2.3s
281:	total: 901ms	remaining: 2.29s
282:	total: 905ms	remaining: 2.29s
283:	total: 917ms	remaining: 2.31s
284:	total: 924ms	remaining: 2.32s
285:	total: 934ms	remaining: 2.33s
286:	total: 941ms	remaining: 2.34s
287:	total: 945ms	remaining: 2.33s
288:	total: 948ms	remaining: 2.33s
289:	total: 952ms	remaining: 2.33s
290:	total: 955ms	remaining: 2.33s
291:	total: 959ms	remaining: 2.32s
292:	total: 962ms	remaining: 2.32s
293:	total: 965ms	remaining: 2.32s
294:	total: 968ms	remaining: 2.31s
295:	total: 971ms	remaining: 2.31s
296:	total: 974ms	remaining: 2.31s
297:	total: 978ms	remaining: 2.3s
298:	total: 981ms	remaining: 2.3s
299:	total: 984ms	remaining: 2.3s
300:	total: 987ms	remaining: 2.29s
301:	total: 995ms	remaining: 2.3s
302:	total: 1s	remaining: 2.3s
303:	total: 1s	remaining: 2.3s
304:	total: 1.01s	remaining: 2.3s


538:	total: 1.79s	remaining: 1.53s
539:	total: 1.79s	remaining: 1.52s
540:	total: 1.79s	remaining: 1.52s
541:	total: 1.79s	remaining: 1.52s
542:	total: 1.8s	remaining: 1.51s
543:	total: 1.8s	remaining: 1.51s
544:	total: 1.8s	remaining: 1.51s
545:	total: 1.81s	remaining: 1.5s
546:	total: 1.81s	remaining: 1.5s
547:	total: 1.81s	remaining: 1.5s
548:	total: 1.82s	remaining: 1.49s
549:	total: 1.82s	remaining: 1.49s
550:	total: 1.82s	remaining: 1.49s
551:	total: 1.83s	remaining: 1.48s
552:	total: 1.83s	remaining: 1.48s
553:	total: 1.83s	remaining: 1.48s
554:	total: 1.84s	remaining: 1.47s
555:	total: 1.84s	remaining: 1.47s
556:	total: 1.84s	remaining: 1.47s
557:	total: 1.85s	remaining: 1.46s
558:	total: 1.85s	remaining: 1.46s
559:	total: 1.85s	remaining: 1.46s
560:	total: 1.85s	remaining: 1.45s
561:	total: 1.86s	remaining: 1.45s
562:	total: 1.87s	remaining: 1.45s
563:	total: 1.87s	remaining: 1.45s
564:	total: 1.87s	remaining: 1.44s
565:	total: 1.88s	remaining: 1.44s
566:	total: 1.88s	remainin

820:	total: 2.7s	remaining: 588ms
821:	total: 2.7s	remaining: 585ms
822:	total: 2.7s	remaining: 581ms
823:	total: 2.71s	remaining: 578ms
824:	total: 2.71s	remaining: 574ms
825:	total: 2.71s	remaining: 571ms
826:	total: 2.71s	remaining: 568ms
827:	total: 2.72s	remaining: 564ms
828:	total: 2.72s	remaining: 561ms
829:	total: 2.72s	remaining: 558ms
830:	total: 2.73s	remaining: 554ms
831:	total: 2.73s	remaining: 551ms
832:	total: 2.73s	remaining: 548ms
833:	total: 2.73s	remaining: 545ms
834:	total: 2.74s	remaining: 541ms
835:	total: 2.74s	remaining: 538ms
836:	total: 2.75s	remaining: 535ms
837:	total: 2.75s	remaining: 531ms
838:	total: 2.75s	remaining: 528ms
839:	total: 2.75s	remaining: 525ms
840:	total: 2.76s	remaining: 521ms
841:	total: 2.76s	remaining: 518ms
842:	total: 2.76s	remaining: 515ms
843:	total: 2.77s	remaining: 512ms
844:	total: 2.77s	remaining: 508ms
845:	total: 2.78s	remaining: 505ms
846:	total: 2.78s	remaining: 502ms
847:	total: 2.78s	remaining: 499ms
848:	total: 2.79s	remai

RandomizedSearchCV(cv=2, error_score='raise-deprecating',
          estimator=<catboost.core.CatBoostClassifier object at 0x000001CA23E24208>,
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'depth': [4, 6, 8, 10, 12], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'iterations': [500, 1000, 1500, 2000, 2500, 3000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [953]:
report(random_search_3.cv_results_,5)

Model with rank: 1
Mean validation score: 0.863 (std: 0.00272)
Parameters: {'learning_rate': 0.02, 'iterations': 1000, 'depth': 4}

Model with rank: 2
Mean validation score: 0.861 (std: 0.00122)
Parameters: {'learning_rate': 0.01, 'iterations': 3000, 'depth': 4}

Model with rank: 3
Mean validation score: 0.859 (std: 0.00463)
Parameters: {'learning_rate': 0.02, 'iterations': 2000, 'depth': 6}

Model with rank: 4
Mean validation score: 0.857 (std: 0.00544)
Parameters: {'learning_rate': 0.05, 'iterations': 1500, 'depth': 4}

Model with rank: 5
Mean validation score: 0.854 (std: 0.00259)
Parameters: {'learning_rate': 0.05, 'iterations': 500, 'depth': 10}



In [960]:
model_best= CatBoostClassifier(class_weights=[1,2],eval_metric='AUC',loss_function="Logloss",random_seed = 23,learning_rate=0.02,iterations=1000,depth=4)

In [961]:
model_best.fit(X_train,Y_train)

0:	total: 3.91ms	remaining: 3.9s
1:	total: 7.65ms	remaining: 3.82s
2:	total: 11.2ms	remaining: 3.73s
3:	total: 14.2ms	remaining: 3.54s
4:	total: 17ms	remaining: 3.39s
5:	total: 20ms	remaining: 3.31s
6:	total: 22.8ms	remaining: 3.23s
7:	total: 25.7ms	remaining: 3.19s
8:	total: 28.8ms	remaining: 3.17s
9:	total: 31.6ms	remaining: 3.13s
10:	total: 34.4ms	remaining: 3.09s
11:	total: 38.3ms	remaining: 3.15s
12:	total: 41.7ms	remaining: 3.16s
13:	total: 45.1ms	remaining: 3.18s
14:	total: 49ms	remaining: 3.22s
15:	total: 51.4ms	remaining: 3.16s
16:	total: 54.3ms	remaining: 3.14s
17:	total: 57ms	remaining: 3.11s
18:	total: 59.9ms	remaining: 3.09s
19:	total: 62.7ms	remaining: 3.07s
20:	total: 65.5ms	remaining: 3.05s
21:	total: 68.4ms	remaining: 3.04s
22:	total: 71.8ms	remaining: 3.05s
23:	total: 75.2ms	remaining: 3.06s
24:	total: 78.3ms	remaining: 3.05s
25:	total: 81.7ms	remaining: 3.06s
26:	total: 85.2ms	remaining: 3.07s
27:	total: 88.6ms	remaining: 3.07s
28:	total: 92.7ms	remaining: 3.1s
29:	t

267:	total: 897ms	remaining: 2.45s
268:	total: 899ms	remaining: 2.44s
269:	total: 903ms	remaining: 2.44s
270:	total: 905ms	remaining: 2.44s
271:	total: 908ms	remaining: 2.43s
272:	total: 911ms	remaining: 2.43s
273:	total: 915ms	remaining: 2.42s
274:	total: 918ms	remaining: 2.42s
275:	total: 921ms	remaining: 2.42s
276:	total: 924ms	remaining: 2.41s
277:	total: 927ms	remaining: 2.41s
278:	total: 930ms	remaining: 2.4s
279:	total: 933ms	remaining: 2.4s
280:	total: 935ms	remaining: 2.39s
281:	total: 938ms	remaining: 2.39s
282:	total: 941ms	remaining: 2.38s
283:	total: 944ms	remaining: 2.38s
284:	total: 948ms	remaining: 2.38s
285:	total: 955ms	remaining: 2.38s
286:	total: 959ms	remaining: 2.38s
287:	total: 962ms	remaining: 2.38s
288:	total: 968ms	remaining: 2.38s
289:	total: 977ms	remaining: 2.39s
290:	total: 984ms	remaining: 2.4s
291:	total: 988ms	remaining: 2.39s
292:	total: 995ms	remaining: 2.4s
293:	total: 998ms	remaining: 2.4s
294:	total: 1s	remaining: 2.39s
295:	total: 1s	remaining: 2.

537:	total: 1.78s	remaining: 1.53s
538:	total: 1.79s	remaining: 1.53s
539:	total: 1.79s	remaining: 1.52s
540:	total: 1.79s	remaining: 1.52s
541:	total: 1.79s	remaining: 1.52s
542:	total: 1.8s	remaining: 1.51s
543:	total: 1.8s	remaining: 1.51s
544:	total: 1.8s	remaining: 1.51s
545:	total: 1.81s	remaining: 1.5s
546:	total: 1.81s	remaining: 1.5s
547:	total: 1.81s	remaining: 1.5s
548:	total: 1.82s	remaining: 1.49s
549:	total: 1.82s	remaining: 1.49s
550:	total: 1.82s	remaining: 1.49s
551:	total: 1.83s	remaining: 1.48s
552:	total: 1.83s	remaining: 1.48s
553:	total: 1.83s	remaining: 1.48s
554:	total: 1.84s	remaining: 1.47s
555:	total: 1.84s	remaining: 1.47s
556:	total: 1.85s	remaining: 1.47s
557:	total: 1.85s	remaining: 1.47s
558:	total: 1.85s	remaining: 1.46s
559:	total: 1.86s	remaining: 1.46s
560:	total: 1.86s	remaining: 1.46s
561:	total: 1.86s	remaining: 1.45s
562:	total: 1.87s	remaining: 1.45s
563:	total: 1.87s	remaining: 1.45s
564:	total: 1.88s	remaining: 1.44s
565:	total: 1.88s	remainin

789:	total: 2.68s	remaining: 712ms
790:	total: 2.68s	remaining: 708ms
791:	total: 2.69s	remaining: 705ms
792:	total: 2.69s	remaining: 702ms
793:	total: 2.69s	remaining: 698ms
794:	total: 2.69s	remaining: 695ms
795:	total: 2.7s	remaining: 692ms
796:	total: 2.7s	remaining: 689ms
797:	total: 2.71s	remaining: 685ms
798:	total: 2.71s	remaining: 682ms
799:	total: 2.71s	remaining: 678ms
800:	total: 2.72s	remaining: 675ms
801:	total: 2.72s	remaining: 672ms
802:	total: 2.72s	remaining: 669ms
803:	total: 2.73s	remaining: 665ms
804:	total: 2.73s	remaining: 663ms
805:	total: 2.74s	remaining: 659ms
806:	total: 2.74s	remaining: 656ms
807:	total: 2.75s	remaining: 653ms
808:	total: 2.75s	remaining: 650ms
809:	total: 2.75s	remaining: 646ms
810:	total: 2.76s	remaining: 643ms
811:	total: 2.76s	remaining: 640ms
812:	total: 2.77s	remaining: 636ms
813:	total: 2.77s	remaining: 633ms
814:	total: 2.77s	remaining: 630ms
815:	total: 2.78s	remaining: 626ms
816:	total: 2.78s	remaining: 623ms
817:	total: 2.78s	rema

In [962]:
p3=model_best.predict_proba(X_test)[:,1]

In [963]:
roc_auc_score(Y_test,p3)

0.832732977607623

In [974]:
sample_submission["gender"]= model_best.predict_proba(testing)[:,1]

In [975]:
sample_submission["gender"]=np.where(sample_submission["gender"]>0.50,"female","male")

In [976]:
sample_submission.to_csv("D:/Python/JantaHack/sample_submission.csv",header =True, index=False)